In [262]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
from tensorflow.keras.models import Sequential, Model, load_model


In [277]:
# define the standalone discriminator model
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1), n_classes=10):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# scale up to image dimensions with linear activation
	n_nodes = in_shape[0] * in_shape[1]
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((in_shape[0], in_shape[1], 1))(li)
	# image input
	in_image = Input(shape=in_shape)
	# concat label as a channel
	merge = Concatenate()([in_image, li])
	# downsample
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# dropout
	fe = Dropout(0.4)(fe)
	# output
	out_layer = Dense(1, activation='sigmoid')(fe)
	# define model
	model = Model([in_image, in_label], out_layer)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [264]:
import tensorflow as tf
data_dir = "F:\\Omar\\labelled dataset\\test\\"

train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    image_size=(64, 64),
    color_mode='rgb',
    batch_size=128,
    shuffle=True
    )

class_names = train_ds.class_names
print(class_names)


Found 132 files belonging to 2 classes.
['green', 'white']


In [265]:
#print(train_ds)
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  


(128, 64, 64, 3)
(128,)
(4, 64, 64, 3)
(4,)


In [273]:
i=0
for images, labels in train_ds.take(-1):
    #image, label = next(iter(train_ds))
    x=image.numpy()
    y=label.numpy()

    
  

In [274]:
print(x)
print(len(y))

[[[[3.75000e-01 1.59125e+02 8.93750e+01]
   [1.87500e+00 1.53500e+02 1.17375e+02]
   [2.15000e+01 1.51750e+02 9.91250e+01]
   ...
   [2.32500e+01 1.49000e+02 1.20375e+02]
   [6.25000e-01 1.36625e+02 9.52500e+01]
   [1.17500e+01 1.33750e+02 6.92500e+01]]

  [[0.00000e+00 1.59500e+02 1.08625e+02]
   [3.12500e+00 1.59500e+02 1.07250e+02]
   [1.07500e+01 1.46125e+02 1.06500e+02]
   ...
   [4.01250e+01 1.41875e+02 9.11250e+01]
   [7.00000e+00 1.44250e+02 1.04375e+02]
   [7.25000e+00 1.45875e+02 1.02375e+02]]

  [[1.67500e+01 1.51125e+02 1.14750e+02]
   [1.02500e+01 1.62500e+02 9.77500e+01]
   [3.50000e+00 1.42375e+02 8.90000e+01]
   ...
   [1.52500e+01 1.45500e+02 1.16625e+02]
   [1.62500e+01 1.46125e+02 9.71250e+01]
   [8.62500e+00 1.34375e+02 8.62500e+01]]

  ...

  [[9.25000e+00 1.47500e+02 8.66250e+01]
   [2.55000e+01 1.47500e+02 1.21375e+02]
   [3.12500e+00 1.52000e+02 1.00250e+02]
   ...
   [1.35000e+01 8.65000e+01 5.75000e+01]
   [2.50000e+00 9.70000e+01 4.97500e+01]
   [4.62500e+00 

In [278]:
# define the standalone generator model
# define the standalone generator model
def define_generator(latent_dim, n_classes=2):
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	n_nodes = 7 * 7
	li = Dense(n_nodes)(li)
	# reshape to additional channel
	li = Reshape((7, 7, 1))(li)
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	gen = Dense(n_nodes)(in_lat)
	gen = LeakyReLU(alpha=0.2)(gen)
	gen = Reshape((7, 7, 128))(gen)
	# merge image gen and label input
	merge = Concatenate()([gen, li])
	# upsample to 14x14
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 28x28
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# output
	out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
	# define model
	model = Model([in_lat, in_label], out_layer)
	return model

In [279]:
def load_real_samples():
	# load dataset
	(trainX, trainy)= x,y
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return [X, trainy]

In [281]:
data=load_real_samples()
print(len(data))

2


In [282]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_label = g_model.input
	# get image output from the generator model
	gen_output = g_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = d_model([gen_output, gen_label])
	# define gan model as taking noise and label and outputting a classification
	model = Model([gen_noise, gen_label], gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model




# # select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator.predict([z_input, labels_input])
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y



# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real)
			# generate 'fake' examples
			[X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake)
			# prepare points in latent space as input for the generator
			[z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
	# save the generator model
	g_model.save('cgan_generator.h5')
    



In [283]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
g_model.summary()
d_model.summary()

# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

Model: "functional_109"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_87 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_86 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense_88 (Dense)                (None, 6272)         633472      input_87[0][0]                   
__________________________________________________________________________________________________
embedding_44 (Embedding)        (None, 1, 50)        100         input_86[0][0]                   
_____________________________________________________________________________________

ValueError: in user code:

    C:\Users\ctj-oe\AppData\Local\conda\conda\envs\Hands\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\ctj-oe\AppData\Local\conda\conda\envs\Hands\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ctj-oe\AppData\Local\conda\conda\envs\Hands\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ctj-oe\AppData\Local\conda\conda\envs\Hands\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ctj-oe\AppData\Local\conda\conda\envs\Hands\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
   

In [284]:
# example of loading the generator model and generating images
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model
from matplotlib import pyplot

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# create and save a plot of generated images
def save_plot(examples, n):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	pyplot.show()

# load model
model = load_model('cgan_generator.h5')

# generate images
latent_points, labels = generate_latent_points(100, 100)
# specify labels
labels = asarray([x for _ in range(10) for x in range(10)])
# generate images
X  = model.predict([latent_points, labels])
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
save_plot(X, 10)

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  indices[2,0] = 2 is not in [0, 2)
	 [[node functional_73/embedding_26/embedding_lookup (defined at <ipython-input-284-7fe277c71970>:38) ]]
	 [[functional_73/embedding_26/embedding_lookup/_10]]
  (1) Invalid argument:  indices[2,0] = 2 is not in [0, 2)
	 [[node functional_73/embedding_26/embedding_lookup (defined at <ipython-input-284-7fe277c71970>:38) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_predict_function_23507]

Errors may have originated from an input operation.
Input Source operations connected to node functional_73/embedding_26/embedding_lookup:
 functional_73/embedding_26/embedding_lookup/23397 (defined at C:\Users\ctj-oe\AppData\Local\conda\conda\envs\Hands\lib\contextlib.py:81)

Input Source operations connected to node functional_73/embedding_26/embedding_lookup:
 functional_73/embedding_26/embedding_lookup/23397 (defined at C:\Users\ctj-oe\AppData\Local\conda\conda\envs\Hands\